<a href="https://colab.research.google.com/github/yrc00/DACON/blob/main/%EC%99%80%EC%9D%B8%20%ED%92%88%EC%A7%88%20%EB%B6%84%EB%A5%98/%EC%99%80%EC%9D%B8%ED%92%88%EC%A7%88%EB%B6%84%EB%A5%98_%EB%AA%A8%EB%8D%B8%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/DACON/와인 분류/data

Mounted at /content/drive
/content/drive/MyDrive/DACON/와인 분류/data


In [ ]:
import pandas as pd
train = pd.read_csv('train.csv').drop(columns='index')
test = pd.read_csv('test.csv').drop(columns='index')
submission = pd.read_csv('sample_submission.csv')

In [ ]:
train.head()

,quality,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type
0,5,5.6,0.695,0.06,6.8,0.042,9.0,84.0,0.99432,3.44,0.44,10.2,white
1,5,8.8,0.610,0.14,2.4,0.067,10.0,42.0,0.99690,3.19,0.59,9.5,red
2,5,7.9,0.210,0.39,2.0,0.057,21.0,138.0,0.99176,3.05,0.52,10.9,white
3,6,7.0,0.210,0.31,6.0,0.046,29.0,108.0,0.99390,3.26,0.50,10.8,white
4,6,7.8,0.400,0.26,9.5,0.059,32.0,178.0,0.99550,3.04,0.43,10.9,white


# 표준화

In [ ]:
from sklearn.preprocessing import StandardScaler

train_scaled = train.drop(columns='quality')
test_scaled = test.copy()
features = train_scaled.columns[0:11]

# scaler
scaler = StandardScaler()
scaler.fit(train_scaled[features])
train_scaled[features] = scaler.transform(train_scaled[features])
test_scaled[features] = scaler.transform(test_scaled[features])

# encoding
type_encoded = pd.get_dummies(train_scaled['type'], drop_first=True)
train_sc_en = train_scaled.drop(columns='type')
train_sc_en = pd.concat([train_sc_en, type_encoded], axis=1)

type_encoded = pd.get_dummies(test_scaled['type'], drop_first=True)
test_sc_en = test_scaled.drop(columns='type')
test_sc_en = pd.concat([test_sc_en, type_encoded], axis=1)

In [ ]:
train_sc_en.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,white
0,-1.250611,2.186377,-1.781940,0.286345,-0.398500,-1.211937,-0.560852,-0.117252,1.372128,-0.605988,-0.255287,1
1,1.234899,1.665574,-1.230560,-0.638755,0.322998,-1.155351,-1.307080,0.738864,-0.183584,0.398147,-0.841348,0
2,0.535849,-0.785265,0.492500,-0.722855,0.034399,-0.532907,0.398583,-0.966732,-1.054782,-0.070450,0.330774,1
3,-0.163201,-0.785265,-0.058879,0.118145,-0.283060,-0.080221,-0.134436,-0.256620,0.252016,-0.204334,0.247051,1
4,0.458177,0.378883,-0.403491,0.854020,0.092119,0.089537,1.109276,0.274305,-1.117010,-0.672931,0.330774,1


In [ ]:
test_sc_en.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,white
0,1.390243,-0.172555,1.112802,0.244295,-0.369640,-1.098765,-0.756293,-0.289803,-1.988209,-1.007643,0.916835,1
1,4.730148,0.562696,1.802026,-0.743880,0.409577,-0.872422,-1.342614,1.900261,-0.992553,-0.271277,-1.259963,0
2,-0.551562,-0.356368,-0.334569,-0.050055,-0.456220,0.768567,1.127044,0.042025,-0.183584,1.067570,-0.925071,1
3,-0.007856,-1.152890,0.492500,-0.764905,-0.369640,-0.532907,0.771697,0.042025,1.869955,-0.405161,-0.422733,1
4,-0.318545,-0.478910,-0.403491,-0.722855,-1.062277,-0.391443,-0.774060,-1.414699,-0.370269,-0.405161,1.084281,1


## 학습

In [ ]:
from sklearn.ensemble import RandomForestClassifier

X_train = train_sc_en.copy()
y_train = train['quality']
X_test = test_sc_en.copy()

rf = RandomForestClassifier(random_state=123)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
submission['quality'] = y_pred
submission.head()

,index,quality
0,0,5
1,1,6
2,2,6
3,3,5
4,4,6


In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(rf, X_train, y_train, cv=5)
print("acc: "+str(scores.mean()))

acc: 0.6734514021010837


In [ ]:
submission.to_csv('rf_scale_base.csv', index=False)

- 0.706

# 표준화 + 정규화

In [ ]:
from sklearn.preprocessing import StandardScaler, Normalizer

train_scaled = train.drop(columns='quality')
test_scaled = test.copy()
features = train_scaled.columns[0:11]

# scaler
scaler = StandardScaler()
scaler.fit(train_scaled[features])
train_scaled[features] = scaler.transform(train_scaled[features])
test_scaled[features] = scaler.transform(test_scaled[features])



train_normalized = train_scaled.copy()
test_normalized = test_scaled.copy()

# normalizer
normal = Normalizer()
normal.fit(train_normalized[features])
train_normalized[features] = normal.transform(train_normalized[features])
test_normalized[features] = normal.transform(test_normalized[features])


# encoding
type_encoded = pd.get_dummies(train_normalized['type'], drop_first=True)
train_nm_en = train_normalized.drop(columns='type')
train_nm_en = pd.concat([train_sc_en, type_encoded], axis=1)

type_encoded = pd.get_dummies(test_normalized['type'], drop_first=True)
test_nm_en = test_normalized.drop(columns='type')
test_nm_en = pd.concat([test_sc_en, type_encoded], axis=1)

In [ ]:
train_nm_en.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,white,white
0,-1.250611,2.186377,-1.781940,0.286345,-0.398500,-1.211937,-0.560852,-0.117252,1.372128,-0.605988,-0.255287,1,1
1,1.234899,1.665574,-1.230560,-0.638755,0.322998,-1.155351,-1.307080,0.738864,-0.183584,0.398147,-0.841348,0,0
2,0.535849,-0.785265,0.492500,-0.722855,0.034399,-0.532907,0.398583,-0.966732,-1.054782,-0.070450,0.330774,1,1
3,-0.163201,-0.785265,-0.058879,0.118145,-0.283060,-0.080221,-0.134436,-0.256620,0.252016,-0.204334,0.247051,1,1
4,0.458177,0.378883,-0.403491,0.854020,0.092119,0.089537,1.109276,0.274305,-1.117010,-0.672931,0.330774,1,1


In [ ]:
test_nm_en.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,white,white
0,1.390243,-0.172555,1.112802,0.244295,-0.369640,-1.098765,-0.756293,-0.289803,-1.988209,-1.007643,0.916835,1,1
1,4.730148,0.562696,1.802026,-0.743880,0.409577,-0.872422,-1.342614,1.900261,-0.992553,-0.271277,-1.259963,0,0
2,-0.551562,-0.356368,-0.334569,-0.050055,-0.456220,0.768567,1.127044,0.042025,-0.183584,1.067570,-0.925071,1,1
3,-0.007856,-1.152890,0.492500,-0.764905,-0.369640,-0.532907,0.771697,0.042025,1.869955,-0.405161,-0.422733,1,1
4,-0.318545,-0.478910,-0.403491,-0.722855,-1.062277,-0.391443,-0.774060,-1.414699,-0.370269,-0.405161,1.084281,1,1


## 학습

In [ ]:
from sklearn.ensemble import RandomForestClassifier

X_train = train_nm_en.copy()
y_train = train['quality']
X_test = test_nm_en.copy()

rf = RandomForestClassifier(random_state=123)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
submission['quality'] = y_pred
submission.head()

,index,quality
0,0,5
1,1,5
2,2,6
3,3,5
4,4,6


In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(rf, X_train, y_train, cv=5)
print("acc: "+str(scores.mean()))

acc: 0.6749087600297792


In [ ]:
submission.to_csv('rf_normalize_base.csv', index=False)

- 0.686

# feature_engineering

## add alcohol, density columns

In [ ]:
import numpy as np

# standardscaler + encoding
train_feature = train_sc_en.copy()
test_feature = test_sc_en.copy()

# alcohol, density split standard
alcohol_Q3 = train_feature['alcohol'].quantile(.75)
density_Q1 = train_feature['density'].quantile(.25)

# train
train_feature['high_alcohol'] = np.where(train_feature['alcohol'] > alcohol_Q3, 1, 0)
train_feature['low_density'] = np.where(train_feature['density'] < density_Q1, 1, 0)

# test
test_feature['high_alcohol']= np.where(test_feature['alcohol'] > alcohol_Q3, 1, 0)
test_feature['low_density'] = np.where(test_feature['density'] < density_Q1, 1, 0)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

X_train = train_feature.copy()
y_train = train['quality']
X_test = test_feature.copy()

rf = RandomForestClassifier(random_state=123)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
submission['quality'] = y_pred
submission.head()

,index,quality
0,0,6
1,1,6
2,2,6
3,3,5
4,4,6


In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(rf, X_train, y_train, cv=5)
print("acc: "+str(scores.mean()))

acc: 0.6727257837703698


In [ ]:
submission.to_csv('rf_feature_base.csv', index=False)

- 0.691

## add alcohol, density columns, alcohol * density

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

# standardscaler + encoding
train_feature = train_sc_en.copy()
test_feature = test_sc_en.copy()

# alcohol, density split standard
alcohol_Q3 = train_feature['alcohol'].quantile(.75)
density_Q1 = train_feature['density'].quantile(.25)

# train
train_feature['high_alcohol'] = np.where(train_feature['alcohol'] > alcohol_Q3, 1, 0)
train_feature['low_density'] = np.where(train_feature['density'] < density_Q1, 1, 0)
train_feature['alcohol*density'] = train_feature['alcohol'] * train_feature['density']

# test
test_feature['alcohol*density'] = test_feature['alcohol'] * test_feature['density']
test_feature['high_alcohol'] = np.where(test_feature['alcohol'] > alcohol_Q3, 1, 0)
test_feature['low_density'] = np.where(test_feature['density'] < density_Q1, 1, 0)

# Reorder columns in test_feature to match the order in train_feature
test_feature = test_feature[train_feature.columns]

X_train = train_feature.copy()
y_train = train['quality']
X_test = test_feature.copy()

rf = RandomForestClassifier(random_state=123)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
submission['quality'] = y_pred
submission.head()

,index,quality
0,0,6
1,1,6
2,2,6
3,3,5
4,4,6


In [ ]:
submission.to_csv('rf_feature_all.csv', index=False)

- 0.687

## add alcohol * density column

In [ ]:
import numpy as np

# standardscaler + encoding
train_feature = train_sc_en.copy()
test_feature = test_sc_en.copy()


# train
train_feature['alcohol*density'] = train_feature['alcohol'] * train_feature['density']

# test
test_feature['alcohol*density'] = test_feature['alcohol'] * test_feature['density']

In [ ]:
from sklearn.ensemble import RandomForestClassifier

X_train = train_feature.copy()
y_train = train['quality']
X_test = test_feature.copy()

rf = RandomForestClassifier(random_state=123)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
submission['quality'] = y_pred
submission.head()

,index,quality
0,0,5
1,1,6
2,2,6
3,3,5
4,4,6


In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(rf, X_train, y_train, cv=5)
print("acc: "+str(scores.mean()))

acc: 0.672361320208454


In [ ]:
submission.to_csv('rf_feature_mul.csv', index=False)

- 0.697

# feature selection

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

X_train = train_sc_en.copy()
y_train = train['quality']
X_test = test_sc_en.copy()

rf = RandomForestClassifier(random_state=123)
rf.fit(X_train, y_train)

sfm = SelectFromModel(rf, threshold='median')
sfm.fit(X_train, y_train)

selected_features = sfm.get_support(indices=True)
X_train_selected = X_train.iloc[:, selected_features]

print("Selected features: ", X_train.columns[selected_features])

Selected features:  Index(['volatile acidity', 'chlorides', 'total sulfur dioxide', 'density',
       'sulphates', 'alcohol'],
      dtype='object')


In [ ]:
from sklearn.ensemble import RandomForestClassifier

features = ['volatile acidity', 'chlorides', 'total sulfur dioxide', 'density',
       'sulphates', 'alcohol']
X_train = train_sc_en[features]
y_train = train['quality']
X_test = test_sc_en[features]

rf = RandomForestClassifier(random_state=123)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
submission['quality'] = y_pred
submission.head()

,index,quality
0,0,6
1,1,6
2,2,6
3,3,5
4,4,6


In [ ]:
submission.to_csv('rf_feature_selection_base.csv', index=False)

- 0.666

## feature engineering dataset

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

X_train = train_feature.copy()
y_train = train['quality']
X_test = test_feature.copy()

rf = RandomForestClassifier(random_state=123)
rf.fit(X_train, y_train)

sfm = SelectFromModel(rf, threshold='median')
sfm.fit(X_train, y_train)

selected_features = sfm.get_support(indices=True)
X_train_selected = X_train.iloc[:, selected_features]

print("Selected features: ", X_train.columns[selected_features])

Selected features:  Index(['volatile acidity', 'free sulfur dioxide', 'total sulfur dioxide',
       'density', 'sulphates', 'alcohol', 'alcohol*density'],
      dtype='object')


In [ ]:
from sklearn.ensemble import RandomForestClassifier

features = ['volatile acidity', 'free sulfur dioxide', 'total sulfur dioxide',
       'density', 'sulphates', 'alcohol', 'alcohol*density']
X_train = train_feature[features]
y_train = train['quality']
X_test = test_feature[features]

rf = RandomForestClassifier(random_state=123)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
submission['quality'] = y_pred
submission.head()

,index,quality
0,0,5
1,1,6
2,2,6
3,3,5
4,4,6


In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(rf, X_train, y_train, cv=5)
print("acc: "+str(scores.mean()))

acc: 0.6603565224584333


In [ ]:
submission.to_csv('rf_feature_engineering_selection.csv', index=False)

- 0.7

# 하이퍼파라미터 튜닝

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

X_train = train_sc_en.copy()
y_train = train['quality']
X_test = test_sc_en.copy()

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'random_state': [42, 123, 456]
}

rf = RandomForestClassifier()
gs = GridSearchCV(estimator=rf, param_grid=param_grid,
                  scoring='accuracy', cv=10, n_jobs=-1)

result = gs.fit(X_train, y_train)
print("최적 점수: {}".format(result.best_score_))
print("최적 파라미터: {}".format(result.best_params_))
print(gs.best_estimator_)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 5 members, which is less than n_splits=10.
  warnings.warn(


최적 점수: 0.6929173704255672
최적 파라미터: {'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300, 'random_state': 42}
RandomForestClassifier(max_depth=20, n_estimators=300, random_state=42)


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

train_scaled = train.drop(columns='quality')
test_scaled = test.copy()
features = train_scaled.columns[0:11]

# scaler
scaler = StandardScaler()
scaler.fit(train_scaled[features])
train_scaled[features] = scaler.transform(train_scaled[features])
test_scaled[features] = scaler.transform(test_scaled[features])

# encoding
type_encoded = pd.get_dummies(train_scaled['type'], drop_first=True)
train_sc_en = train_scaled.drop(columns='type')
train_sc_en = pd.concat([train_sc_en, type_encoded], axis=1)

type_encoded = pd.get_dummies(test_scaled['type'], drop_first=True)
test_sc_en = test_scaled.drop(columns='type')
test_sc_en = pd.concat([test_sc_en, type_encoded], axis=1)

X_train = train_sc_en.copy()
y_train = train['quality']
X_test = test_sc_en.copy()

rf = RandomForestClassifier(max_depth=20, min_samples_leaf=1, min_samples_split=2, n_estimators=300, random_state=42)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
submission['quality'] = y_pred
submission.head()

,index,quality
0,0,6
1,1,6
2,2,6
3,3,5
4,4,6


In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(rf, X_train, y_train, cv=5)
print("acc: "+str(scores.mean()))

acc: 0.6798218214906113


In [ ]:
submission.to_csv('rf_grid_base.csv', index=False)

- 0.694

# 최종 모델

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

train_scaled = train.drop(columns='quality')
test_scaled = test.copy()
features = train_scaled.columns[0:11]

# scaler
scaler = StandardScaler()
scaler.fit(train_scaled[features])
train_scaled[features] = scaler.transform(train_scaled[features])
test_scaled[features] = scaler.transform(test_scaled[features])

# encoding
type_encoded = pd.get_dummies(train_scaled['type'], drop_first=True)
train_sc_en = train_scaled.drop(columns='type')
train_sc_en = pd.concat([train_sc_en, type_encoded], axis=1)

type_encoded = pd.get_dummies(test_scaled['type'], drop_first=True)
test_sc_en = test_scaled.drop(columns='type')
test_sc_en = pd.concat([test_sc_en, type_encoded], axis=1)

# training
X_train = train_sc_en.copy()
y_train = train['quality']
X_test = test_sc_en.copy()

rf = RandomForestClassifier(random_state=123)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
submission['quality'] = y_pred
submission.head()